In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
from urllib.parse import urljoin

os.chdir('C:/Python/PGIM_Intern/ETF Data Transfer')

In [ ]:


# 登入資訊
LOGIN_URL = "https://cleartest.twse.com.tw/etf/logon.jsp"
FILE_DOWNLOAD_URL = "https://cleartest.twse.com.tw/etf/FSys_Logon"
ETF_ID = "0098"  # 指定的 ETF ID
USER_ID = "PGIM8802"
PASSWORD = "Aa202410"
DOWNLOAD_DIR = "ETF_Data"  # 公槽的實際路徑

def ensure_directory_exists(directory):
    """檢查並建立目錄"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"目錄 {directory} 已建立。")
    else:
        print(f"目錄 {directory} 已存在。")

def login_and_download():
    # 建立 Session
    session = requests.Session()

    # 登入資料
    login_data = {
        'etfId': "PGIM88",  # 公司 ID
        'userId': USER_ID,  # 用戶 ID
        'password': PASSWORD,  # 密碼
    }

    # 模擬登入
    response = session.post(LOGIN_URL, data=login_data)
    if response.status_code != 200 or "錯誤" in response.text:
        print("登入失敗，請檢查登入資訊或網站狀態。")
        return

    print("登入成功！正在嘗試下載指定檔案...")

    # 確保下載目錄存在
    ensure_directory_exists(DOWNLOAD_DIR)

    # Step 2: Access the logged-in page to find the file link
    response = session.get(FILE_DOWNLOAD_URL)
    if response.status_code != 200:
        print("無法取得檔案頁面，請檢查網站狀態。")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # Step 3: Find the table and locate the download link
    rows = soup.find_all('tr', class_='tableData')
    for row in rows:
        # Find the specific <div> containing 'M12'
        div = row.find('td', width="100").find('div', align="center")
        if div and div.text.strip() == 'M12':
            # If 'M12' is found, locate the download link
            download_td = row.find('td', width="258")
            if download_td:
                strong_tag = download_td.find('strong')
                if strong_tag:
                    download_link = strong_tag.find('a')['href']
                    # Construct the full download URL
                    download_url = urljoin(FILE_DOWNLOAD_URL, download_link)

                    # Step 4: Download the file
                    download_file(session, download_url)
                    break

def download_file(session, download_url):
    """下載指定檔案"""
    print(f"檔案下載 URL：{download_url}")

    # 發起下載請求
    response = session.get(download_url, stream=True)
    if response.status_code == 200:
        file_name = download_url.split('=')[-1]
        filepath = os.path.join(DOWNLOAD_DIR, file_name)
        with open(filepath, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"檔案已成功下載並儲存至：{filepath}")
    else:
        print(f"下載檔案失敗，HTTP 狀態碼：{response.status_code}")
        print("請確認檔案是否存在或權限是否足夠。")

if __name__ == "__main__":
    print("開始執行下載程式...")
    login_and_download()


開始執行下載程式...
登入成功！正在嘗試下載指定檔案...
目錄 ETF_Data 已建立。
